In [1]:
import numpy as np
import pandas as pd
import theano
import theanets # autoencoders
from sklearn.cross_validation import train_test_split
from scipy import misc
import math

In [2]:
def crop(img):
    top, bottom, left, right = 200, -131, 100, -100
    return img[top:bottom, left:right]

In [3]:
def distance(c1, c2):
    (r1,g1,b1) = c1
    (r2,g2,b2) = c2
    return math.sqrt((r1 - r2)**2 + (g1 - g2) ** 2 + (b1 - b2) **2)

In [32]:
hit_map = {(255, 255, 255) : 0, # white nothing
           (0, 0, 0) : 0,       # homerun. not defensible
           (255, 0, 0) : 1,     # linedrive, laser
           (0, 255, 0) : 2,     # groundball  
           (0, 0, 255) : 3,     # flyball
           (160, 32, 240) : 4}  # blooper

colors = hit_map.keys()
    
def norm_color(rgb):
    rgb_key = tuple(rgb)
    
    if rgb_key in colors:
        return hit_map[rgb_key]
    else:    
        sc = sorted(colors, key=lambda color: distance(color, rgb))
        return hit_map[sc[0]]

In [33]:
mb = crop(misc.imread('charts/pros/Mookie_Betts.png', mode='RGB'))

In [35]:
mb = [[norm_color(rgb) for rgb in row] for row in mb]

In [37]:
np.unique(np.array(mb))

array([0, 1, 2, 3, 4])